In [1]:
import random
import math
import csv
import copy

In [2]:
class Item:
    def __init__(self, charid, depth, width, height, weight, ptype, profit, itemid):
        #assuming that along x axis is width, along y axis is height and along z axis is depth/length
        self.itemid = int(itemid)
        self.width = int(width)
        self.height = int(height)
        self.depth = int(depth)
        self.profit = int(profit)
        self.weight = int(weight)
        self.charid = charid
        self.x = None
        self.y = None
        self.z = None
        self.ptype = ptype
        
    def __repr__(self):
        return f"Item({self.itemid}, {self.width}, {self.height}, {self.depth}, ({self.x}, {self.y}, {self.z}), {self.weight}, {self.profit}, {self.ptype})\n"


In [3]:
class Uld:
    #assuming that along x axis is width, along y axis is height and along z axis is depth/length
    def __init__(self, charid, depth, width, height, capacity, uid):
        self.uid = int(uid)
        self.capacity = int(capacity)
        self.width = int(width)
        self.height = int(height)
        self.depth = int(depth)
        self.charid = charid
    def __repr__(self):
        return f"ULD({self.charid}, {self.depth}, {self.width}, {self.height}, {self.capacity})"

In [4]:
def createSequenceTable(A,B,C):
    n = len(A)
    dictA = {}
    for i in range(n):
        dictA[A[i]] = i+1
    
    dictB = {}
    for i in range(n):
        dictB[B[i]] = i+1

    dictC = {}
    for i in range(n):
        dictC[C[i]] = i+1
    
    table = [dictA, dictB, dictC]
    return table

In [5]:
def canPlaceItemInUld(item, uld):
    if item.width>uld.width or item.height>uld.height or item.depth>uld.depth:
        return False
    if item.weight>uld.capacity:
        return False
    
    return True

In [6]:
def itemIndex(tup, seq_index):
    item_id = tup[0]
    table = tup[1]
    dict_seq = table[seq_index]
    
    return dict_seq[item_id]

In [13]:
def transformTo3D(uld, i, placed_items_tot, table):
    px,py,pz=[],[],[]
    #checking each placed item with my current item
    placed_items = []
    for items in placed_items_tot:
        if(items[0]==uld.uid):
            placed_items.append(items)
    for tup in placed_items:
        item_placed = tup[1]
        item = (i, table)
        j = (item_placed.itemid, table)
        #j is item placed and item is item to be placed
        if((itemIndex(item,0)<itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)>itemIndex(j,2))):
            px.append(tup[1])
        if((itemIndex(item,0)>itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)<itemIndex(j,2))):
            py.append(tup[1])
        if(((itemIndex(item,0)>itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)>itemIndex(j,2)))\
             or ((itemIndex(item,0)<itemIndex(j,0)) and (itemIndex(item,1)<itemIndex(j,1)) and (itemIndex(item,2)<itemIndex(j,2)) )):
            pz.append(tup[1])
    dimx,dimy,dimz=0,0,0
    for j in px:
        dimx = max(dimx, j.x + j.width)
    for j in py:
        dimy = max(dimy, j.y + j.height)
    for j in pz:
        dimz = max(dimz, j.z + j.depth)
        
    stable = False
    if(len(placed_items)==0):
        stable = True
    for tup in placed_items:
        item = tup[1]
        if(dimx>=item.x and dimx<=(item.x+item.width) and dimz>=item.z and dimz<=(item.z + item.depth)):
            stable = True
            break
    
    if(stable==False):
        #making it out of box so that it is invalid
        dimx = uld.width+1
    
    x = max(0,dimx)
    y = max(0,dimy)
    z = max(0,dimz)
    
    return x,y,z

In [14]:
def packUld(original_pos, A, B, C, items, ulds, n, table):
    
    #n is the number of items
    placed_items = original_pos[:]
    score = 0
    total_weight = [0]*len(ulds)
    total_profit = 0
    for item_index_seq in range(n-1,-1,-1):
        curItem = items[B[item_index_seq]-1]
        placed = False
        for uld in ulds:
            x,y,z = transformTo3D(uld, curItem.itemid, placed_items, table)
            if(x+curItem.width <= uld.width and\
               y+curItem.height <= uld.height and\
               z+curItem.depth <= uld.depth and\
               curItem.weight + total_weight[uld.uid - 1] <= uld.capacity
              ):
                curItem.x, curItem.y , curItem.z = x,y,z
                placed_items.append([uld.uid,curItem])
                score += curItem.profit
                total_profit += curItem.profit
                total_weight[uld.uid - 1] += curItem.weight
                placed = True
                break
        if not placed:
            #penalty for unplaced items
            total_profit -= 0.5*curItem.profit
                
    return score, total_profit, placed_items

In [15]:
def modifySequence(A,B,C):
    new_A = A[:]
    new_B = B[:]
    new_C = C[:]
    swap_ch = random.choice([0,1])
    if(swap_ch == 1):
        seq_choice = random.choice(['A','B','C'])

        if seq_choice == 'A':
            i,j = random.sample(range(len(A)), 2)
            new_A[i], new_A[j] = new_A[j], new_A[i]

        elif seq_choice == 'B':
            i,j = random.sample(range(len(A)), 2)
            new_B[i], new_B[j] = new_B[j], new_B[i]

        elif seq_choice == 'C':
            i,j = random.sample(range(len(A)), 2)
            new_C[i], new_C[j] = new_C[j], new_C[i]
    else:
        i,j = random.sample(range(len(A)), 2)
        new_A[i], new_A[j] = new_A[j], new_A[i]
        
        i,j = random.sample(range(len(A)), 2)
        new_B[i], new_B[j] = new_B[j], new_B[i]
        
        i,j = random.sample(range(len(A)), 2)
        new_C[i], new_C[j] = new_C[j], new_C[i]
        
    return new_A,new_B,new_C

In [16]:
def simulateAnnealing(original_pos, items, ulds, max_iteration = 1000, temp = 1100, decrease_factor = 0.999):
    n = len(items)
    total_iterations = 1
    current_A = list(range(1,n+1))
    current_B = list(range(1,n+1))
    current_C = list(range(1,n+1))
    random.shuffle(current_A)
    random.shuffle(current_B)
    random.shuffle(current_C)
    table = createSequenceTable(current_A, current_B, current_C)
    items_copy = copy.deepcopy(items)
    ulds_copy = copy.deepcopy(ulds)
    score, current_profit, placed_items = packUld(original_pos, current_A, current_B, current_C, items_copy, ulds_copy, n, table)
    #using slicing to avoid same reference
    best_A , best_B, best_C =  current_A[:], current_B[:], current_C[:]
    best_placing = copy.deepcopy(placed_items)
    best_profit = current_profit
    best_score = score
    T = temp
    while(score < 30000):
        print("initial score", score)
        current_A = list(range(1,n+1))
        current_B = list(range(1,n+1))
        current_C = list(range(1,n+1))
        random.shuffle(current_A)
        random.shuffle(current_B)
        random.shuffle(current_C)
        table = createSequenceTable(current_A, current_B, current_C)

        items_copy = copy.deepcopy(items)
        ulds_copy = copy.deepcopy(ulds)
        score, current_profit, placed_items = packUld(original_pos, current_A, current_B, current_C, items_copy, ulds_copy, n, table)
        #using slicing to avoid same reference
        best_A , best_B, best_C =  current_A[:], current_B[:], current_C[:]
        best_placing = copy.deepcopy(placed_items)
        best_profit = current_profit
        best_score = score
        T = temp
        total_iterations += 1
    

    while(True):
        total_iterations += 1
        new_A, new_B, new_C = modifySequence(current_A, current_B, current_C)
        table = createSequenceTable(new_A, new_B, new_C)
        items_copy = copy.deepcopy(items)
        ulds_copy = copy.deepcopy(ulds)
        new_score, new_profit, placed_items = packUld(original_pos, new_A, new_B, new_C, items_copy, ulds_copy, n, table)
        delta = (new_profit-current_profit)
        
        if delta>0 or random.uniform(0,1) < math.exp(delta/T):
            current_A , current_B, current_C = new_A[:], new_B[:], new_C[:]
            current_profit = new_profit
            #if the solution is accepted with some probabilty then we do not update our best profits
            if new_profit>best_profit:
                best_A, best_B, best_C = new_A[:], new_B[:], new_C[:]
                best_score = new_score
                best_profit = new_profit
                best_placing = placed_items[:]
            
            T *= decrease_factor
            
        #print(table)
        print(f'temperature:{T}, best-score:{best_score}, new-score={new_score}, count={len(best_placing)}')
        print("---------------------------------------")
        if T < 1e-3:
            break
            
    print("Temperature :",T)
    print("Score :",best_score)
    print("total iterations :", total_iterations)
    return best_placing, best_score

In [17]:
#extracting data from file

uld_obj = open('ulds.csv', 'r')
item_obj = open('packages.csv', 'r')
item_data = csv.reader(item_obj)
#items_eco=[]
#items_prio=[]
items = []
ulds=[]
total_cost = 0
priority_count = 0
ind = 0
for row in item_data:
    if ind == 0:
        ind = 1
        continue
    row = [x.strip().replace("'", "").strip() for x in row]  # Remove quotes
    ptype = row[5]
    if(ptype=="Economy"):
        total_cost += int(row[6])
        row.append(ind) 
        item = Item(*row) 
        #items_eco.append(item)
        items.append(item)
        #ind+=1
     
    else:
        priority_count +=1
        row[6] = 1000
        row.append(ind)
        item = Item(*row)
        #items_prio.append(item)
        items.append(item)
        
    ind+=1
        
uld_item = csv.reader(uld_obj)
ind = 0
for row in uld_item:
    if(ind==0):
        ind = 1
        continue
    row = [x.strip().replace("'", "").strip() for x in row]  # Remove quotes
    row.append(ind)
    container = Uld(*row) 
    ulds.append(container)
    ind+=1
print(total_cost)
print("priority Count :", priority_count)
print(items)
uld_obj.close()
item_obj.close()

29233
priority Count : 103
[Item(1, 53, 55, 99, (None, None, None), 61, 176, Economy)
, Item(2, 99, 81, 56, (None, None, None), 53, 1000, Priority)
, Item(3, 101, 51, 42, (None, None, None), 17, 1000, Priority)
, Item(4, 75, 56, 108, (None, None, None), 73, 138, Economy)
, Item(5, 58, 64, 88, (None, None, None), 93, 139, Economy)
, Item(6, 56, 84, 91, (None, None, None), 47, 1000, Priority)
, Item(7, 78, 93, 88, (None, None, None), 117, 102, Economy)
, Item(8, 105, 76, 108, (None, None, None), 142, 108, Economy)
, Item(9, 71, 88, 73, (None, None, None), 50, 1000, Priority)
, Item(10, 70, 85, 88, (None, None, None), 81, 1000, Priority)
, Item(11, 80, 81, 55, (None, None, None), 23, 96, Economy)
, Item(12, 80, 88, 48, (None, None, None), 27, 117, Economy)
, Item(13, 94, 87, 55, (None, None, None), 41, 73, Economy)
, Item(14, 46, 81, 45, (None, None, None), 27, 68, Economy)
, Item(15, 49, 60, 84, (None, None, None), 57, 1000, Priority)
, Item(16, 93, 63, 48, (None, None, None), 82, 1000, 

In [ ]:
'''#sample data
items = [
    item(itemid=1, height=10, width=5, depth=20, weight=15, profit=100),
    item(itemid=2, height=15, width=7, depth=12, weight=10, profit=80),
    item(itemid=3, height=8, width=6, depth=18, weight=12, profit=90),
    item(itemid=4, height=12, width=5, depth=15, weight=8, profit=70),
    item(itemid=5, height=7, width=4, depth=10, weight=5, profit=40),
]

# Create a list of ULDs
ulds = [
    uld(uid=1, width=20, height=30, depth=20, capacity=200),
    #uld(uid=2, width=10, height=20, depth=20, capacity=300),
]

'''
noitem = 0
original_pos  = []
arrangement, score = simulateAnnealing(original_pos, items, ulds, temp = 1200)
noitem = len(arrangement)
print("length:",noitem)
print(arrangement)

initial score 25236
initial score 20731
initial score 21744
initial score 20001
initial score 17584
initial score 21284
initial score 20442
initial score 18396
initial score 24457
initial score 14568
initial score 16709
initial score 18987
initial score 25275
initial score 18045
initial score 21044
initial score 21028
initial score 18878
initial score 17056
initial score 26518
initial score 18079
initial score 19236
initial score 20825
initial score 21188
initial score 27080
initial score 22233
initial score 16752
initial score 22281
initial score 10773
initial score 20388
initial score 18125
initial score 24375
initial score 27773
initial score 25958
initial score 23960
initial score 18377
initial score 17396
initial score 22979
initial score 13754
initial score 22765
initial score 15714
initial score 14979
initial score 21638
initial score 24205
initial score 18323
initial score 19517
initial score 20732
initial score 19778
initial score 22753
initial score 18640
initial score 22493


initial score 20602
initial score 18039
initial score 24361
initial score 25745
initial score 22260
initial score 18715
initial score 22367
initial score 23002
initial score 22725
initial score 18064
initial score 23024
initial score 21652
initial score 23093
initial score 20965
initial score 19487
initial score 21863
initial score 17603
initial score 22267
initial score 25339
initial score 17576
initial score 21125
initial score 21052
initial score 22747
initial score 18328
initial score 17417
initial score 21842
initial score 25051
initial score 22988
initial score 17736
initial score 22935
initial score 24358
initial score 24830
initial score 21823
initial score 19325
initial score 15845
initial score 26388
initial score 23049
initial score 19004
initial score 26543
initial score 19941
initial score 20182
initial score 17455
initial score 16721
initial score 21290
initial score 19580
initial score 20049
initial score 21293
initial score 15392
initial score 17149
initial score 19900


temperature:1154.078331214189, best-score:35314, new-score=35314, count=77
---------------------------------------
temperature:1154.078331214189, best-score:35314, new-score=23979, count=77
---------------------------------------
temperature:1152.9242528829748, best-score:35314, new-score=35314, count=77
---------------------------------------
temperature:1152.9242528829748, best-score:35314, new-score=28621, count=77
---------------------------------------
temperature:1151.7713286300918, best-score:36124, new-score=36124, count=76
---------------------------------------
temperature:1150.6195573014618, best-score:36124, new-score=36124, count=76
---------------------------------------
temperature:1149.4689377441603, best-score:36124, new-score=36124, count=76
---------------------------------------
temperature:1148.3194688064161, best-score:36124, new-score=36057, count=76
---------------------------------------
temperature:1148.3194688064161, best-score:36124, new-score=33129, count=7

temperature:1109.9139954817722, best-score:37933, new-score=36933, count=76
---------------------------------------
temperature:1108.8040814862904, best-score:38080, new-score=38080, count=78
---------------------------------------
temperature:1108.8040814862904, best-score:38080, new-score=37253, count=78
---------------------------------------
temperature:1108.8040814862904, best-score:38080, new-score=23832, count=78
---------------------------------------
temperature:1108.8040814862904, best-score:38080, new-score=27978, count=78
---------------------------------------
temperature:1107.695277404804, best-score:38080, new-score=38080, count=78
---------------------------------------
temperature:1107.695277404804, best-score:38080, new-score=24821, count=78
---------------------------------------
temperature:1106.5875821273992, best-score:38080, new-score=38080, count=78
---------------------------------------
temperature:1105.4809945452719, best-score:38080, new-score=38080, count=7

temperature:1069.5778291612228, best-score:38080, new-score=35717, count=78
---------------------------------------
temperature:1068.5082513320615, best-score:38080, new-score=34789, count=78
---------------------------------------
temperature:1068.5082513320615, best-score:38080, new-score=32771, count=78
---------------------------------------
temperature:1068.5082513320615, best-score:38080, new-score=28218, count=78
---------------------------------------
temperature:1068.5082513320615, best-score:38080, new-score=26574, count=78
---------------------------------------
temperature:1067.4397430807294, best-score:38080, new-score=34789, count=78
---------------------------------------
temperature:1066.3723033376486, best-score:38080, new-score=34789, count=78
---------------------------------------
temperature:1065.305931034311, best-score:38080, new-score=34987, count=78
---------------------------------------
temperature:1064.2406251032767, best-score:38080, new-score=35917, count=

...

temperature:4.035898433429977, best-score:61912, new-score=61912, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=61912, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=32561, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=60018, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=42956, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=45904, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=41107, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=49884, count=97
---------------------------------------
temperature:4.031862534996547, best-score:61912, new-score=56864, count=97
-----

temperature:3.9244057075383942, best-score:61912, new-score=36159, count=97
---------------------------------------
temperature:3.9244057075383942, best-score:61912, new-score=58993, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=61912, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=60972, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=59712, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=29003, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=36628, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=48838, count=97
---------------------------------------
temperature:3.920481301830856, best-score:61912, new-score=57978, count=97
---

temperature:3.8312952050700595, best-score:61912, new-score=55041, count=97
---------------------------------------
temperature:3.8312952050700595, best-score:61912, new-score=49108, count=97
---------------------------------------
temperature:3.8312952050700595, best-score:61912, new-score=55834, count=97
---------------------------------------
temperature:3.8312952050700595, best-score:61912, new-score=48901, count=97
---------------------------------------
temperature:3.8274639098649894, best-score:61912, new-score=61912, count=97
---------------------------------------
temperature:3.8274639098649894, best-score:61912, new-score=53572, count=97
---------------------------------------
temperature:3.8274639098649894, best-score:61912, new-score=60912, count=97
---------------------------------------
temperature:3.8274639098649894, best-score:61912, new-score=40471, count=97
---------------------------------------
temperature:3.8236364459551244, best-score:61912, new-score=61912, count

temperature:3.7291838795427323, best-score:61912, new-score=52197, count=97
---------------------------------------
temperature:3.7291838795427323, best-score:61912, new-score=57965, count=97
---------------------------------------
temperature:3.7291838795427323, best-score:61912, new-score=52378, count=97
---------------------------------------
temperature:3.7291838795427323, best-score:61912, new-score=58912, count=97
---------------------------------------
temperature:3.7291838795427323, best-score:61912, new-score=37029, count=97
---------------------------------------
temperature:3.7254546956631898, best-score:61912, new-score=61912, count=97
---------------------------------------
temperature:3.7217292409675267, best-score:61912, new-score=61912, count=97
---------------------------------------
temperature:3.7217292409675267, best-score:61912, new-score=56117, count=97
---------------------------------------
temperature:3.718007511726559, best-score:61912, new-score=61912, count=

temperature:3.651649209115962, best-score:61917, new-score=55874, count=97
---------------------------------------
temperature:3.651649209115962, best-score:61917, new-score=30583, count=97
---------------------------------------
temperature:3.651649209115962, best-score:61917, new-score=30878, count=97
---------------------------------------


In [59]:
#original_pos = arrangement
#arrangement, score = simulateAnnealing(original_pos, items_eco, ulds, temp = 1100)
#eco_item_count = len(arrangement)
#print(arrangement)

In [61]:

total_packages = len(arrangement)
eco_item_count = 0
print("total packages :", total_packages)
eco_cost = 0
prio_uld_set = set()
for i in range(total_packages):
    if(arrangement[i][1].ptype == "Economy"):
        eco_cost += arrangement[i][1].profit
        eco_item_count += 1
    else:
        prio_uld_set.add(arrangement[i][0])
K = 5000
prio_cost = K * len(prio_uld_set)
net_cost = prio_cost + total_cost - eco_cost
net_cost = prio_cost
print("ULDs used in priority packages :",prio_uld_set)
print("Priority Cost (assuming K = 5000): ", prio_cost)  
print("Count of priority packages sent :",total_packages - eco_item_count)
print("Count of Economy Packages sent:", eco_item_count)
print("economy score :", eco_cost)
print("Economy package delayed Cost :", total_cost - eco_cost)




#print("TOTAL COST : ",net_cost)


total packages : 139
ULDs used in priority packages : {1, 2, 3, 4, 5, 6}
Priority Cost (assuming K = 5000):  30000
Count of priority packages sent : 73
Count of Economy Packages sent: 66
economy score : 6906
Economy package delayed Cost : 22327


In [62]:
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
#original_pos = arrangement
#arrangement, score = simulateAnnealing(original_pos, items_eco, ulds, temp = 1100)
#eco_item_count = len(arrangement)
def visualize_solution(bins, elev = 0, azim = 0, uldno=1):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for i in range(len(arrangement)):
        if(arrangement[i][0]!=uldno):continue
        item = arrangement[i][1]
        ax.bar3d(
            item.x,  # x-coordinate of the bar
            item.y,  # y-coordinate of the bar
            item.z,  # z-coordinate of the bar
            item.width,  # width of the bar
            item.height,  # depth of the bar
            item.depth,  # height of the bar
            shade=True,
        )
        ax.text(item.x, item.z , item.y + item.depth/2, f'{item.itemid}', color='black')

    ax.set_xlabel('Width')
    ax.set_ylabel('Height')
    ax.set_zlabel('Depth')
    #elev = 90  # Elevation angle (degrees)
    #azim = 0  # Azimuthal angle (degrees)
    ax.view_init(elev=elev, azim=azim)
    plt.show()
    

vertical_angle = 0
horizontal_angle = 0
while(True):
    clear_output(wait=True)
    visualize_solution(arrangement, vertical_angle, horizontal_angle, 1)
    horizontal_angle = (horizontal_angle + 5)%180
    vertical_angle = (vertical_angle + 5)%180

KeyboardInterrupt: 

In [40]:
#saving output to file

opfile = open("solution_final_1.csv", 'w', newline = '')
writer = csv.writer(opfile)
writer.writerow([net_cost, total_packages, len(prio_uld_set)])
for row in arrangement:
    itemobj = row[1]
    uid = row[0]
    charid = itemobj.charid
    x0 = itemobj.x
    y0 = itemobj.y
    z0 = itemobj.z
    x1 = x0 + itemobj.width
    y1 = y0 + itemobj.height
    z1 = z0 + itemobj.depth
    writer.writerow([charid, "U"+str(uid), z0, x0, y0, z1, x1, y1])
    
opfile.close()